In [52]:
# Import the data from the Excel Sheet "Data.xlsx" 
 
## The suggested libraries are given, but feel free to import any others that you are comfortable with

In [64]:
# Импорт необходимых библиотек
import pandas as pd
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.io as pio
import mpld3
from mpl_to_plotly import plot
from pfrom mpl_to_plotly import plotlotly.subplots import make_subplots

In [65]:
## Import the data (Only the indices), and create the Year-Over-Year dataframe

In [84]:
# Импорт данных из файла Excel
df = pd.read_excel('Data.xlsx', sheet_name='Data', skiprows=1)

# Задаем названия столбцов в соответствии данными
df.columns = ['Date', 'CN: CPI: MoM: Food, Tobacco & Liquor', 'CN: Agricultural Product Price: Wholesale: Meat: Fresh Pork', 'WD FAO FOOD PRICE INDEX (NOMINAL) NADJ', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']

# Преобразуем столбцы 'Date' в формат даты
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')

# Подсчет годового изменения Food CPI
df['Food CPI YoY'] = df['CN: CPI: MoM: Food, Tobacco & Liquor'].pct_change(12) * 100

# Вывод первых строк DataFrame
df.head()


/tmp/ipykernel_590/2650980707.py:11: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



,Date,"CN: CPI: MoM: Food, Tobacco & Liquor",CN: Agricultural Product Price: Wholesale: Meat: Fresh Pork,WD FAO FOOD PRICE INDEX (NOMINAL) NADJ,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Food CPI YoY
0,2016-01-01,101.400000,23.406,84.861238,NaN,NaN,NaN,NaN,NaN
1,2016-02-01,106.064400,24.510,86.046383,NaN,NaN,NaN,NaN,NaN
2,2016-03-01,104.685563,24.810,87.422205,NaN,NaN,NaN,NaN,NaN
3,2016-04-01,103.743393,25.542,89.177346,NaN,NaN,NaN,NaN,NaN
4,2016-05-01,101.876012,26.355,90.583839,NaN,NaN,NaN,NaN,NaN


In [85]:
## Create a regression 
### Find the optimal variables, leads and lags, et. cetera. (No right or wrong answer here)

In [108]:
import numpy as np
# Очищаем данные от NA значений и бесконечностей
df = df.replace([ np.inf, -np.inf], np.nan)
df = df.dropna()

# Определяем зависимую и независимые переменные для регрессии
y = df['CN: CPI: MoM: Food, Tobacco & Liquor']
X = df[['CN: Agricultural Product Price: Wholesale: Meat: Fresh Pork', 'WD FAO FOOD PRICE INDEX (NOMINAL) NADJ']]

# Добавьте константу (intercept) в независимые переменные
X = sm.add_constant(X)

# Выполните регрессию
model = sm.OLS(y, X).fit()

# Выведите статистику регрессии
print(model.summary())

                                     OLS Regression Results                                     
Dep. Variable:     CN: CPI: MoM: Food, Tobacco & Liquor   R-squared:                       0.818
Model:                                              OLS   Adj. R-squared:                  0.814
Method:                                   Least Squares   F-statistic:                     173.4
Date:                                  Mon, 30 Oct 2023   Prob (F-statistic):           3.05e-29
Time:                                          21:00:58   Log-Likelihood:                -218.61
No. Observations:                                    80   AIC:                             443.2
Df Residuals:                                        77   BIC:                             450.4
Df Model:                                             2                                         
Covariance Type:                              nonrobust                                         
                              

In [93]:
## Make a forecast using the betas of the regressors and use the "extra data" available to insert the data into the FoodCPI column 
### Hint: Use pd.DateOffset

In [126]:
# Получаем оцененные коэффициенты из регрессии
beta0 = model.params['const']
beta1 = model.params['CN: Agricultural Product Price: Wholesale: Meat: Fresh Pork']
beta2 = model.params['WD FAO FOOD PRICE INDEX (NOMINAL) NADJ']

# Значения независимых переменных для прогноза
fresh_pork_price = df['CN: Agricultural Product Price: Wholesale: Meat: Fresh Pork']  # Используйте уже имеющиеся значения из вашего DataFrame
food_price_index = df['WD FAO FOOD PRICE INDEX (NOMINAL) NADJ']  # Используйте уже имеющиеся значения из вашего DataFrame

# Делаем прогноз
forecast = beta0 + beta1 * fresh_pork_price + beta2 * food_price_index

# прогнозное значение
df['FoodCPI'] = forecast

print(df.head())


         Date  CN: CPI: MoM: Food, Tobacco & Liquor  \
12 2017-01-01                            104.008424   
13 2017-02-01                            103.696398   
14 2017-03-01                            102.348345   
15 2017-04-01                            102.041300   
16 2017-05-01                            101.633135   

    CN: Agricultural Product Price: Wholesale: Meat: Fresh Pork  \
12                                            24.4550             
13                                            24.1425             
14                                            22.8780             
15                                            22.1850             
16                                            21.3525             

    WD FAO FOOD PRICE INDEX (NOMINAL) NADJ           Unnamed: 4 Unnamed: 5  \
12                               97.702431  2017-01-01 00:00:00   0.025724   
13                               98.119070  2017-02-01 00:00:00  -0.022326   
14                              

In [120]:
## Plot the forecast and the actual data on independent y axes as shown
### You can move the axes around by clicking and dragging

In [190]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

# Извлекаем дату, фактические данные и прогноз
date = df['Date']
actual_data = df['CN: CPI: MoM: Food, Tobacco & Liquor']
forecast_data = df['FoodCPI']

# Создаем график Matplotlib
plt.figure(figsize=(12, 6))
plt.plot(date, actual_data, label='Фактические данные', marker='o')
plt.plot(date, forecast_data, linestyle='--', label='Прогнозные данные')
plt.xlabel('Дата')
plt.ylabel('Данные')
plt.title('Chinese Food CPI and Forecast')
plt.legend()
plt.show()
plt.close()
# Создаем график Plotly
fig = px.line(df, x='Date', y=['CN: CPI: MoM: Food, Tobacco & Liquor', 'FoodCPI'], labels={'variable': 'Данные'})
fig.update_traces(line=dict(dash='dash'), selector=dict(name='FoodCPI'))
fig.update_layout(title='Chinese Food CPI and Forecast', xaxis_title='Дата', yaxis_title='Данные')

# Сохраняем график Plotly в HTML
pio.write_html(fig, file='Chinese_Food_CPI_and_Forecast.html')



In [80]:
## Plot the Food CPI and Pork Prices, ensuring that there are dashed lines for the forecasts

In [191]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import plotly.io as pio
import mpld3# Создаем график Matplotlib
plt.figure(figsize=(12, 6))
plt.plot(date, food_cpi, label='Food CPI', marker='o')
plt.plot(date, pork_prices, label='Pork Prices', marker='o')
plt.plot(forecast_date, food_cpi_forecast, linestyle='--', label='Food CPI Forecast')
plt.xlabel('Дата')
plt.ylabel('Значения')
plt.title('Food CPI and Pork Prices')
plt.legend()
plt.show()
plt.close()

# Сохраняем график Matplotlib в HTML
mpld3_fig = mpld3.fig_to_html(plt.gcf())

with open('Food_CPI_and_Pork_Prices.html', 'w') as f:
    f.write(mpld3_fig)

# Создаем график Plotly
fig_plotly = px.line(df, x='Date', y=['CN: CPI: MoM: Food, Tobacco & Liquor', 'FoodCPI'], labels={'variable': 'Данные'})
fig_plotly.update_traces(line=dict(dash='dash'), selector=dict(name='FoodCPI'))
fig_plotly.update_layout(title='Chinese Food CPI and Forecast', xaxis_title='Дата', yaxis_title='Данные')

# Сохраняем график Plotly в HTML
pio.write_html(fig_plotly, file='Chinese_Food_CPI_and_Forecast.html')
